# 最大熵模型

最大熵模型(maximum entropy model)可以用于二分类，也可以用于多分类。其是由最大熵原理推导实现的，所以讲最大熵模型时，绕不开最大熵原理。

### 最大熵原理

什么是最大熵原理？
最大熵原理认为，学习概率模型时，在所有可能的概率模型（概率分布）中，熵最大的模型就是最好的模型。最大熵原理通常表述为在满足约束条件的模型集合中选取熵最大的模型。

假设离散随机变量$X$的概率分布是$P(X)$,则其熵为：
$$H(P)=-\sum_{x}P(x)\log P(x)$$
且满足下列不等式:
$${0 \leq H(P) \leq \log \left|X \right|}$$

上式中，$\left|X\right|$是$X$的取值个数，右边等号成立的条件是$X$的分布是均匀分布。这就说明当$X$服从均匀分布时，熵能取得最大值。

### 最大熵模型

最大熵模型是由以下条件概率分布表示的分类模型：
$${P_w(y|x)=\frac{1}{Z_{w}(x)}\exp \left(\sum_{i=1}^{n} w_i f_i(x,y)\right)}$$

$${Z_w(x)=\sum_{y}\exp \left(\sum_{i=1}^{n} w_i f_i(x.y)\right)}$$
其中，$Z_w(x)$是规范化因子，$f_i$为特征函数，$w_i$为特征的权值。

最大熵原理应用到分类模型学习中，由以下约束最优化问题：

$${\min -H(P)=\sum{x,y}\tilde{P}(x)P(y|x)\log P(y|x)}$$

$${s.t \quad P(f_i)- \tilde(P)(f_i)=0,\quad i=1,2,...,n}$$

$${\sum_{y}P(y|x)=1}$$

求解上述最优化问题的对偶问题便可得到最大熵模型。

In [1]:
import math
from copy import deepcopy

In [2]:
class MaxEntropy:
    def __init__(self, EPS=0.005):
        self._samples = []
        self._label_y = set()  # 标签集合，相当去去重后的y
        self._numXY = {}  # key为(x,y)，value为出现次数
        self._samples_num = 0  # 样本数
        self._Ep_ = []  # 样本分布的特征期望值
        self._xyID = {}  # key记录(x,y),value记录id号
        self._xy_num = 0  # 特征键值(x,y)的个数
        self._max_feature_num = 0  # 最大特征数
        self._IDxy = {}  # key为(x,y)，value为对应的id号
        self._weights = []
        self._EPS = EPS  # 收敛条件
        self._last_weights = []  # 上一次w参数值

    def loadData(self, dataset):
        self._samples = deepcopy(dataset)
        for items in self._samples:
            y = items[0]
            X = items[1:]
            self._label_y.add(y)  # 集合中y若已存在则会自动忽略
            for x in X:
                if (x, y) in self._numXY:
                    self._numXY[(x, y)] += 1
                else:
                    self._numXY[(x, y)] = 1

        self._samples_num = len(self._samples)
        self._xy_num = len(self._numXY)
        self._max_feature_num = max([len(sample) - 1 for sample in self._samples])
        self._weights = [0] * self._xy_num
        self._last_weights = self._weights[:]

        self._Ep_ = [0] * self._xy_num
        for i, xy in enumerate(self._numXY):  # 计算特征函数fi关于经验分布的期望
            self._Ep_[i] = self._numXY[xy] / self._samples_num
            self._xyID[xy] = i
            self._IDxy[i] = xy
    
    # 计算每个Z(x)值
    def _calc_zx(self, X):
        zx = 0
        for y in self._label_y:
            temp = 0
            for x in X:
                if (x, y) in self._numXY:
                    temp += self._weights[self._xyID[(x, y)]]
            zx += math.exp(temp)
        return zx
    
    # 计算每个P(y|x)
    def _calu_model_pyx(self, y, X):  
        zx = self._calc_zx(X)
        temp = 0
        for x in X:
            if (x, y) in self._numXY:
                temp += self._weights[self._xyID[(x, y)]]
        pyx = math.exp(temp) / zx
        return pyx
    
    # 计算特征函数fi关于模型的期望
    def _calc_model_ep(self, index):  
        x, y = self._IDxy[index]
        ep = 0
        for sample in self._samples:
            if x not in sample:
                continue
            pyx = self._calu_model_pyx(y, sample)
            ep += pyx / self._samples_num
        return ep
    
     # 判断是否全部收敛
    def _convergence(self): 
        for last, now in zip(self._last_weights, self._weights):
            if abs(last - now) >= self._EPS:
                return False
        return True
     
    #计算预测概率
    def predict(self, X):  
        Z = self._calc_zx(X)
        result = {}
        for y in self._label_y:
            ss = 0
            for x in X:
                if (x, y) in self._numXY:
                    ss += self._weights[self._xyID[(x, y)]]
            pyx = math.exp(ss) / Z
            result[y] = pyx
        return result
    
    #训练
    def train(self, maxiter=1000): 
        for loop in range(maxiter): 
            print("迭代次数:%d" % loop)
            self._last_weights = self._weights[:]
            for i in range(self._xy_num):
                ep = self._calc_model_ep(i)  # 计算第i个特征的模型期望
                self._weights[i] += math.log(self._Ep_[i] / ep) / self._max_feature_num  # 更新参数
            print("权值:", self._weights)
            if self._convergence():  # 判断是否收敛
                break

In [3]:
dataset = [['no', 'sunny', 'hot', 'high', 'FALSE'],
           ['no', 'sunny', 'hot', 'high', 'TRUE'],
           ['yes', 'overcast', 'hot', 'high', 'FALSE'],
           ['yes', 'rainy', 'mild', 'high', 'FALSE'],
           ['yes', 'rainy', 'cool', 'normal', 'FALSE'],
           ['no', 'rainy', 'cool', 'normal', 'TRUE'],
           ['yes', 'overcast', 'cool', 'normal', 'TRUE'],
           ['no', 'sunny', 'mild', 'high', 'FALSE'],
           ['yes', 'sunny', 'cool', 'normal', 'FALSE'],
           ['yes', 'rainy', 'mild', 'normal', 'FALSE'],
           ['yes', 'sunny', 'mild', 'normal', 'TRUE'],
           ['yes', 'overcast', 'mild', 'high', 'TRUE'],
           ['yes', 'overcast', 'hot', 'normal', 'FALSE'],
           ['no', 'rainy', 'mild', 'high', 'TRUE']]

In [4]:
maxent = MaxEntropy()
x = ['overcast', 'mild', 'high', 'FALSE']

In [5]:
maxent.loadData(dataset)
maxent.train(maxiter=100)

迭代次数:0
权值: [0.0455803891984887, -0.002832177999673058, 0.031103560672370825, -0.1772024616282862, -0.0037548445453157455, 0.16394435955437575, -0.02051493923938058, -0.049675901430111545, 0.08288783767234777, 0.030474400362443962, 0.05913652210443954, 0.08028783103573349, 0.1047516055195683, -0.017733409097415182, -0.12279936099838235, -0.2525211841208849, -0.033080678592754015, -0.06511302013721994, -0.08720030253991244]
迭代次数:1
权值: [0.11525071899801315, 0.019484939219927316, 0.07502777039579785, -0.29094979172869884, 0.023544184009850026, 0.2833018051925922, -0.04928887087664562, -0.101950931659509, 0.12655289130431963, 0.016078718904129236, 0.09710585487843026, 0.10327329399123442, 0.16183727320804359, 0.013224083490515591, -0.17018583153306513, -0.44038644519804815, -0.07026660158873668, -0.11606564516054546, -0.1711390483931799]
迭代次数:2
权值: [0.18178907332733973, 0.04233703122822168, 0.11301330241050131, -0.37456674484068975, 0.05599764270990431, 0.38356978711239126, -0.0748854616816

In [6]:
print('精度:', maxent.predict(x))

精度: {'no': 0.0015600198042872487, 'yes': 0.9984399801957127}


**上述案例均已通过**

----
参考代码：  
[1] [fuqiuai实现最大熵模型](https://github.com/fuqiuai/lihang_algorithms)

整理制作：深度学习学研社

<div>
<table align="left" border="1" bordercolor="#000000">
    <div>
    <tr>
        <td>
            微信公众号：ID: AI_class_vip<br>
            <img src="../image/gongzhonghao.jpg" width="150" height="150" align="left"/>    
        </td>
    </tr>
    </div>
    <div>
    <tr>
        <td>
        知识星球：机器学习交流学习圈：<br>
    <img src="../image/dlzhishixingqiu.jpg" width="150" height="150" align="left"/>  
        </td>
    </tr>
        </div>
    <div>
     <tr>
        <td>
        配置环境：python 3.4+  
        </td>
    </tr>
        </div>
</table>
</div>
